# CRAWLING

개발자 도구 탭 
- Elements - HTML 구조
- Console - Javascript 디버깅
- Sources - 웹페이지를 구성하는 src
- Performance - 웹페이지 성능 체크
- Network - 웹페이지에서 요청한 파일
- Memory - 웹페이지 메모리 사용률
- Application - 브라우저 스토리지정보 (Storage, Session, Cookie)

STATUS CODE
- 200 정상 작동
- 301 리다이렉트
- 404 사용자 오류
- 502 서버 오류

cf. urlencoding 

## 동적 웹 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
headers = {'User-Agent' :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

In [ ]:
response = requests.get("https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2=259", 
                        headers=headers 
                        # params={
                        #     'mode':'LS2D'
                        #     'mid' : 'shm',
                        #     'sid1' : 101,
                        #     'sid2' :259
                        # }
                        )
# requests.post()
# requests.put()
# requests.delete()  요청에 따라 다양한 메서드 사용 가능

✔ ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
headers 정보가 없으면 봇으로 인식하고 뜨는 에러 -> user-agent 정보로 headers 추가하기

In [ ]:
print(response.status_code)


In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

- div 공백으로 나뉘어져 있으면 각각 다른 클래스
- ex) list_body newsflash_body -> list_body.newsflash_body 로 변경 필수

In [ ]:
# newsDivTag = soup.select_one('div.list_body')
newsDivTag = soup.select_one('div.list_body.newsflash_body') # list_body newsflash_body -> list_body.newsflash_body 로 변경 필수

In [ ]:
newsDivTag

In [ ]:
liTags = newsDivTag.select('li')
liTags

In [ ]:
len(liTags)

In [ ]:
liTags[0].select('dt')

In [ ]:
liTags[0].select('dt')[-1].text

In [ ]:
liTags[0].select('dt')[-1].text.strip()

In [ ]:
titles = []
for liTag in liTags:
    dtTag = liTag.select('dt')[-1]
    title = dtTag.text.strip()
    href = dtTag.select_one('a').attrs['href']
    # title = liTag.select('dt')[-1].text.strip() # ('dt')[1]로 설정할 경우 사진이 없는 기사도 있기 때문에 IndexError: list index out of range 에러남 -> ('dt')[-1] 로 변경
    # href = liTag.select('dt')[-1].select_one('a').attrs['href']
    print(href)
    titles.append(title)
titles

---

select 와 select_one 구분하기
- select : list 로 추출
- select_one : 일치하는 하나의 요소만 추출

In [ ]:
dtTag.select('a') 

In [ ]:
dtTag.select('a')[0].attrs['href']

In [ ]:
dtTag.select_one('a').attrs['href']

ID 셀렉터 - id 속성은 unique해서 주로 select_one 사용. #으로 접근
CLASS 셀렉터 - 동일한 요소를 가지고 있는 경우에 CLASS 로 묶으면 편집이 용이함(.사용)

---

## 실습

> 실습 1
- HTML 문서 내에 ID가 mw-content-text인 태그내의 내용을 출력해주세요.
- https://ko.wikipedia.org/wiki/위키백과

In [ ]:
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC' # url encoding되어 있음
response = requests.get(url)
response # 동작이 잘 되는지 확인

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
result = soup.select_one('#mw-content-text').text  # id가 mw-content-text인 text 추출. id 셀렉터를 사용할 때는 select_one 활용하기

In [ ]:
result

> 실습 2
- HTML 문서 내에 class가 list_item인 태그내의 내용을 출력해주세요
- https://www.saramin.co.kr/zf_user/jobs/public/list

In [ ]:
url = "https://www.saramin.co.kr/zf_user/jobs/public/list"
response = requests.get(url, headers=headers)
response

In [ ]:
print(response.text)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
items = soup.select('.list_item')  # class 이름이 list_item
items[0]

In [ ]:
for item in items:
    print(item.text)

- 속성 셀렉터
    - 태그 내의 속성을 활용
    - 셀렉터[속성=“값“], 정확히 일치
    - 셀렉터[속성~=“값”], 해당 단어를 포함
    - 셀렉터[속성^=“값“], 해당 값으로 시작
    - 셀렉터[속성$=“값“], 해당 값으로 끝
    - 셀렉터[속성*=“값”], 해당 값을 포함
- 자식 셀렉터 : '>' 사용  [class^='type06']>li
- 후손 셀렉터 : 공백 사용  [class^='type06'] li

---

## 실습

> 실습 1
- 사이트 내 공지사항을 하나씩 출력해주세요.
- https://didimteo.startup-plus.kr/default.do


In [ ]:
import requests
url = "https://didimteo.startup-plus.kr/cms_for_bcb/process/notice/list.do?show_no=2326&check_no=2312&c_relation=35&c_relation2=23"
response = requests.get(url, headers=headers)
response

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
trTags = soup.select('tbody>tr')  # thead>tr은 제외할 수 있도록

In [ ]:
for trTag in trTags:
    print(trTag.select_one('.list-title').text.strip())

> 실습 2
- 실습 1의 결과에서 제목과 날짜를 분리해 딕셔너리 형태로 저장해주세요
- 결과 : [{'제목': '[채용] 광운대학교 산학협력단(서울창업디딤터) 직원 채용 공고(~6/17)', '날짜': '2022.06.10'},
{'제목': '[서울창업디딤터] 대강의실 및 공동작업실 이용 신청 방식 변경', '날짜': '2022.06.04’}, ... ]

In [ ]:
for trTag in trTags:
    print(trTag.select_one('.list-title').text.strip())
    print(trTag.select_one('.list-right').text.strip())

In [ ]:
result = []
for trTag in trTags:
    temp = {
        '제목' : trTag.select_one('.list-title').text.strip(),
        '날짜' : trTag.select_one('.list-right').text.strip()
    }
    result.append(temp)
result

> 실습 3
- 사이트에서 하이퍼링크에 baCategory1=basic이 포함된 태그를 하나만 선택해 출력해주세요.
- https://youth.seoul.go.kr/site/main/home

In [ ]:
url = 'https://youth.seoul.go.kr/site/main/home'
response = requests.get(url, headers=headers)
response

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
items = soup.select("[href*='baCategory1=basic']")

In [ ]:
for item in items:
    print(item.text.strip())

> 실습 4
- 사이트에서 주요뉴스 내용과 일자별 뉴스들을 리스트로 정리해주세요
- https://ko.wikipedia.org/wiki/포털:요즘_화제
- 결과 : [‘주요 뉴스’, '8월 25일, 파키스탄에서 일어난 홍수로 1,000명 이상의 주민이 사망하고, 가축 70,000마리 이상이 죽었다.’, ..., 
'Current events of 2022년 8월 29일\xa0(2022-08-29) (월요일)', ‘대한민국의 0시 기준 누적 \u200b확진자 수가 23,026,960명으로
집계되었다. 전날 0시 대비 43,142명(국내 42,782, 해외유입 360)이 늘었다.’, ...]

In [ ]:
url = "https://ko.wikipedia.org/wiki/%ED%8F%AC%ED%84%B8:%EC%9A%94%EC%A6%98_%ED%99%94%EC%A0%9C"
response = requests.get(url, headers=headers)
response

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
tableTags = soup.select("[class='vevent']") # soup.select('.vevent')

In [ ]:
len(tableTags)

In [ ]:
mainNews = ['주요 뉴스']
tableTags[0]   # 주요 뉴스 

liTags = tableTags[0].select('li')
for li in liTags:
    mainNews.append(li.text.strip())

mainNews


In [ ]:
resutl = [mainNews]
for tableTag in tableTags[1:-3]:  # 일자별 뉴스 반복문 처리
    result.append([tableTag.select_one('.summary').text.strip(),    # list extend와 list append 차이
                  tableTag.select_one('.description').text.strip()])
result

---